## importing dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import xlsxwriter
import math
import pandas_datareader.data as web
import schwabdev
import os
import dotenv
%matplotlib inline

## importing S&P 500 constituents 

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [4]:
sp500_constituents = sp500_constituents['Symbol'].to_list()
sp500_constituents

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


### changing symbol format to match Schwab's

In [34]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]
sp500_constituents

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK/B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF/B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


## Setting Up Schwab API

In [4]:
from key import api_key
from key import api_secret

In [8]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret, capture_callback = False)

## Getting Quote Data for all constiuents
- want to get price and market cap
    - use .quotes to get latest price 
    - use .intruments to get market cap

### creating Pandas dataframe

In [59]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap' , 'Number of Shares to Buy']

### appending final_df with desired data

In [43]:
entries = []
for stock in sp500_constituents:
    price = client.quotes(stock).json()[stock]['quote']['lastPrice']
    market_cap = client.instruments(stock, projection='fundamental').json()['instruments'][0]['fundamental']['marketCap']
    entries.append([stock, price, market_cap, 'N/a'])

In [44]:
entries

[['MMM', 136.67, 73320328187.0, 'N/a'],
 ['AOS', 64.42, 9257624563.0, 'N/a'],
 ['ABT', 125.96, 220050954387.0, 'N/a'],
 ['ABBV', 177.275, 309659647563.0, 'N/a'],
 ['ACN', 289.06, 178004183080.0, 'N/a'],
 ['ADBE', 352.88, 150222714000.0, 'N/a'],
 ['AMD', 96.55, 150962188554.0, 'N/a'],
 ['AES', 10.375, 7332575634.0, 'N/a'],
 ['AFL', 108.63, 58309373718.0, 'N/a'],
 ['A', 103.2691, 29282900063.0, 'N/a'],
 ['APD', 267.84, 59848178910.0, 'N/a'],
 ['ABNB', 114.535, 71168237669.0, 'N/a'],
 ['AKAM', 74.44, 11131013540.0, 'N/a'],
 ['ALB', 54.54, 6667257688.0, 'N/a'],
 ['ARE', 79.21, 13314218333.0, 'N/a'],
 ['ALGN', 163.73, 12080317942.0, 'N/a'],
 ['ALLE', 127.235, 10799237427.0, 'N/a'],
 ['LNT', 62.0, 15552991364.0, 'N/a'],
 ['ALL', 195.96, 51030765542.0, 'N/a'],
 ['GOOGL', 156.52, 1927959820000.0, 'N/a'],
 ['GOOG', 159.02, 1927959820000.0, 'N/a'],
 ['MO', 57.72, 95775981962.0, 'N/a'],
 ['AMZN', 180.49, 1961907732125.0, 'N/a'],
 ['AMCR', 9.405, 13475722167.0, 'N/a'],
 ['AEE', 99.535, 26323956757

In [62]:
final_df = pd.DataFrame(entries, columns=my_columns)

In [64]:
final_df

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,MMM,136.670,7.332033e+10,N/a
1,AOS,64.420,9.257625e+09,N/a
2,ABT,125.960,2.200510e+11,N/a
3,ABBV,177.275,3.096596e+11,N/a
4,ACN,289.060,1.780042e+11,N/a
...,...,...,...,...
498,XYL,109.520,2.653969e+10,N/a
499,YUM,145.450,4.038499e+10,N/a
500,ZBRA,229.810,1.158293e+10,N/a
501,ZBH,98.110,2.037411e+10,N/a


## Calculating Number of Shares to Buy

In [72]:
portfolio_size = input("Enter the value of the portfolio: ")

try:
    val = float(portfolio_size)
except ValueError:
    print("That is not a number! \n Please try again")
    portfolio_size = (input("Enter the value of the portfolio: "))
    val = float(portfolio_size)

In [74]:
portfolio_size = val/len(final_df.index)

for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(portfolio_size/final_df.loc[i, 'Stock Price'])


In [75]:
final_df

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,MMM,136.670,7.332033e+10,1
1,AOS,64.420,9.257625e+09,3
2,ABT,125.960,2.200510e+11,1
3,ABBV,177.275,3.096596e+11,1
4,ACN,289.060,1.780042e+11,0
...,...,...,...,...
498,XYL,109.520,2.653969e+10,1
499,YUM,145.450,4.038499e+10,1
500,ZBRA,229.810,1.158293e+10,0
501,ZBH,98.110,2.037411e+10,2


## Creating Excel Output

In [89]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

### Formating

In [90]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [91]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [92]:
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats

In [93]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

In [94]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [96]:
writer.close()